# DH307 : R & D Project	

### - Anup Nagdeve | 200010011

## Define Functions

In [10]:
import geopandas as gpd
import os
import pandas as pd
# import folium
import numpy as np
from fastkml import kml
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
import seaborn as sns
from ipywidgets import interact_manual, IntSlider, VBox

import time


In [11]:
def parse_kml(kml_file):
    tree = ET.parse(kml_file)
    root = tree.getroot()
    
    district_data = []
    for placemark in root.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
        district_name = placemark.findtext('.//{http://www.opengis.net/kml/2.2}name')
        extended_data = placemark.find('.//{http://www.opengis.net/kml/2.2}ExtendedData')
        district_id = extended_data.findtext('.//{http://www.opengis.net/kml/2.2}SimpleData[@name="DISTRICT_L"]')
        state = extended_data.findtext('.//{http://www.opengis.net/kml/2.2}SimpleData[@name="STATE"]')
        
        coordinates = placemark.find('.//{http://www.opengis.net/kml/2.2}coordinates').text.strip()
        coordinates_list = [tuple(map(float, coord.split(','))) for coord in coordinates.split()]
        polygon = Polygon(coordinates_list)
        
        district_data.append({
            'district_name': district_name,
            'district_id': district_id,
            'state': state,
            'geometry': polygon
        })
    
    return pd.DataFrame(district_data)

# Parse KML data
maharashtra_districts = parse_kml('maharashtra.kml')


In [12]:
def load_dataset(districts=maharashtra_districts, folder_path='patient_data/'):
    files = os.listdir(folder_path)

    csv_files = [file for file in files if file.endswith('.csv')]

    dataset = {}

    # Load each .csv file using a for loop
    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        df = pd.read_csv(file_path)
        df = df[~df['organisationunitname'].str.contains('CIRCLE', case=False)]
        df['organisationunitname'] = df['organisationunitname'].str.replace('DISTRICT', '', case=False).str.strip()
        # df['organisationunitname'] = df['organisationunitname'].str.upper().str.replace('DISTRICT', '').str.replace('CIRCLE', '').str.strip()
        df.loc[df['organisationunitname'] == 'NASIK', 'organisationunitname'] = 'NASHIK'
        df = df.groupby('organisationunitname').sum().reset_index()
        df.rename(columns={'organisationunitname': 'district_name',
                       'newly diagnosed patients-cvds': 'cvds',
                       'newly diagnosed patients-copd': 'copd',
                       'newly diagnosed patients-ckd': 'ckd',
                       'newly diagnosed patients-nfald': 'nfald',
                       'newly diagnosed patients-stroke': 'stroke',
                       'newly diagnosed patients-rheumatoid arthritis': 'rheumatoid_arthritis'}, inplace=True)
    # Keep only the specified columns and drop others
        df = df[['district_name', 'cvds', 'copd', 'ckd', 'nfald', 'stroke', 'rheumatoid_arthritis']]
        # print(df)
        
        df.loc[df['district_name'] == 'AHMADNAGAR', 'district_name'] = 'AHAMADNAGAR'
        df.loc[df['district_name'] == 'AMRAVATI', 'district_name'] = 'AMARAVATI'
        df.loc[df['district_name'] == 'MUMBAI', 'district_name'] = 'MUMBAI CITY'
        df.loc[df['district_name'] == 'OSMANABAD', 'district_name'] = 'USMANABAD'
        df.loc[df['district_name'] == 'RAIGAD', 'district_name'] = 'RAYGAD'
        df.loc[df['district_name'] == 'YEOTMAL', 'district_name'] = 'YAVATMAL'
        # df = df[df['district_name'] != 'MAHARASHTRA']
        district_mapping = maharashtra_districts[['district_name', 'district_id']]
        merege_data = pd.merge(df, district_mapping, left_on='district_name', right_on='district_name', how='left')
        dataset[csv_file] = merege_data

    return dataset

In [33]:
def plot_map(dataset,quarter_no=1,disease_name='stroke'):
    if quarter_no == 0:
        dfs = [df for df in list(dataset.values())]
        combined_df = pd.concat(dfs)
        disease_data = combined_df.groupby(['district_name', 'district_id']).sum().reset_index()
    else:
        quarter_name = ""
        for k in dataset:
            if int(k[0]) == quarter_no:
                quarter_name = k
        disease_data = dataset[quarter_name]
    # Merge disease data with district geometries
    merged_data = pd.merge(maharashtra_districts, disease_data, on='district_id')

    # Convert DataFrame to GeoDataFrame
    gdf = gpd.GeoDataFrame(merged_data, geometry='geometry')

    # Plot the GeoDataFrame
    fig, ax = plt.subplots(figsize=(10, 8))
    gdf.plot(column=disease_name, cmap='viridis', edgecolor='black', legend=True, ax=ax)

    # Annotate district names on the plot
    for idx, row in gdf.iterrows():
        ax.annotate(text=row['district_name_x'], xy=(row['geometry'].centroid.x, row['geometry'].centroid.y),
                    xytext=(3, 3), textcoords="offset points", fontsize=8,color='white',
                    bbox=dict(facecolor='black', alpha=0.5, edgecolor='none'))

    # Set plot title and labels
    if quarter_no == 0:
        plt_title = f'No. of Patients of {disease_name} All Quarters'
    else:
        plt_title = f'No. of Patients of {disease_name} ({quarter_name[:-4]})'

    plt.title(plt_title)
    plt.xlabel('')
    plt.ylabel('')

    # Show plot
    plt.show()
    
    

In [14]:
def plot_patient_temporal(data_dict, district_name):
    # Initialize figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))
    fig2, ax2 = plt.subplots(figsize=(10, 6))
    bar_width = 0.1  # Adjust as needed

    # Plot line graphs for each disease
    diseases = ['cvds', 'copd', 'ckd', 'nfald', 'stroke','rheumatoid_arthritis']
    quaters = list(data_dict.keys())
    quaters.sort()
    index = np.arange(len(quaters)-1)
    for i, disease in enumerate(diseases):
        patients = [data_dict[quarter][data_dict[quarter]['district_name'] == district_name][disease].iloc[0] 
                    if district_name in data_dict[quarter]['district_name'].values else 0
                    for quarter in quaters]

        ax.plot(quaters, patients, label=disease)

        gr = [patients[i+1]-patients[i] for i in range(len(quaters)-1)]
        # ax2.plot(quaters[:-1], gr, label=disease)
        ax2.bar(index + i * bar_width, gr, bar_width, label=disease)
        

    dfs = [data_dict[quarter] for quarter in quaters]
    combined_df = pd.concat(dfs, ignore_index=True)

    # Filter out rows where the district name is not 'YourDistrictName'
    filtered_df = combined_df[combined_df['district_name'] == district_name]

    # Select only the columns representing diseases
    disease_data = filtered_df[diseases]

# Calculate correlation matrix
    correlation_matrix = disease_data.corr()

    # Plot correlation matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", annot_kws={"size": 10})
    plt.title('Correlation Matrix of Diseases')
    
    # fig2, ax2 = plt.subplots(figsize=(10, 6))
    # plt.figure(figsize=(10, 8))
    # plt.plot(quaters[:-1],growth_rates)
    # plt.title('Growth Rates per Qauter')
    # plt.ylabel('Growth Rates (%)')
    # plt.xlabel('Quaters')
    # plt.xticks(range(len(data_dict)-1), labels=quaters[:-1])

    # Add title and labels
    ax.set_title(f'Patient Counts for {district_name} by Disease')
    ax.set_xlabel('Quarters')
    ax.set_ylabel('Number of Patients')
    ax.set_xticks(range(len(data_dict)))
    ax.set_xticklabels(quaters, rotation=45)
    ax.legend()
    plt.tight_layout()

    ax2.set_xlabel('Quarters')
    ax2.set_ylabel('Growth Rate')
    ax2.set_title('Patients Growth Rate by Disease (Bar Graph)')
    ax2.set_xticks(index + bar_width * (len(diseases) - 1) / 2)
    ax2.set_xticklabels(quaters[:-1],rotation=45)
    ax2.legend()
    ax2.grid(True)
    plt.tight_layout()

    # ax2.set_title(f'Patient Growth Rate for {district_name} by Disease')
    # ax2.set_xlabel('Quarters')
    # ax2.set_ylabel('Growth Rate of Patients')
    # ax2.set_xticks(range(len(data_dict)-1))
    # ax2.set_xticklabels(quaters[:-1], rotation=45)
    # ax2.legend()
    # plt.tight_layout()
    plt.show()


In [21]:
def create_interactive_plot_temporal(data_dict):
    district_names = data_dict[next(iter(data_dict))]['district_name'].unique()

    @interact_manual(district_name=district_names)
    def plot_interactive(district_name):
        plot_patient_temporal(data_dict, district_name)

def create_interactive_plot(dataset):
    quarter_options = list(dataset.keys())
    disease_options = list(dataset[quarter_options[0]].columns)[1:]
    quarter_no = [int(i[0]) for i in quarter_options]+ [0,]
    quarter_no.sort()
    all_quaters_no = quarter_no

    q = quarter_no[0]

    @interact_manual(quarter_no=(quarter_no[0], quarter_no[-1]), disease_name=disease_options)
    def plot_interactive(disease_name, quarter_no=q):
        plot_map(dataset, quarter_no, disease_name)
            

## Spatial Data Analysis

In [22]:
dataset = load_dataset()

In [34]:
create_interactive_plot(dataset)


interactive(children=(Dropdown(description='disease_name', options=('cvds', 'copd', 'ckd', 'nfald', 'stroke', …

## Temporal Data Analysis : District Wise

In [18]:
create_interactive_plot_temporal(dataset)

interactive(children=(Dropdown(description='district_name', options=('AHAMADNAGAR', 'AKOLA', 'AMARAVATI', 'AUR…